In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
model = SiameseDuplicateImageNetwork().to(device)

train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8, persistent_workers=True)

test_dataset = DuplicateImageDataset('../data/Test Data', transforms=[Resize((224, 224), antialias=True)])
test_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8, persistent_workers=True)

print(model)

SiameseDuplicateImageNetwork(
  (preprocess): ImageClassification(
      crop_size=[224]
      resize_size=[232]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BILINEAR
  )
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): MobileNetV3(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): 

In [8]:
len(train_dataloader)

1556

In [9]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(
    [params for params in model.parameters() if params.requires_grad],
    lr=1e-3,
    alpha=0.9
)

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train_loop(train_dataloader, loss_fn, optimizer, device)
    model.test_loop(test_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 0.663543  [  256/398161]
loss: 0.103814  [25856/398161]
loss: 0.086094  [51456/398161]
loss: 0.024835  [77056/398161]
loss: 0.048310  [102656/398161]
loss: 0.081009  [128256/398161]
loss: 0.069121  [153856/398161]
loss: 0.041196  [179456/398161]
loss: 0.156612  [205056/398161]
loss: 0.047120  [230656/398161]
loss: 0.088390  [256256/398161]
loss: 0.005031  [281856/398161]
loss: 0.178339  [307456/398161]
loss: 0.071937  [333056/398161]
loss: 0.024164  [358656/398161]
loss: 0.069734  [384256/398161]
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.080941 

Epoch 2
-------------------------------
loss: 0.015570  [  256/398161]
loss: 0.045508  [25856/398161]
loss: 0.428809  [51456/398161]
loss: 0.040033  [77056/398161]
loss: 0.394351  [102656/398161]
loss: 0.044352  [128256/398161]
loss: 0.426213  [153856/398161]
loss: 0.099505  [179456/398161]
loss: 0.011963  [205056/398161]
loss: 0.046073  [230656/398161]
loss: 0.026053  [256256/398161]
loss: 0.0213

KeyboardInterrupt: 